In [18]:
# 載入需要的套件
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import sys
import datetime
from bs4 import BeautifulSoup

options = Options()
options.add_argument("--disable-notifications")

In [19]:
# a=input()

In [20]:
website = 'https://web.pcc.gov.tw'

In [21]:
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [22]:
try:
    # 開啟瀏覽器視窗(Chrome)
    # 方法一：執行前需開啟chromedriver.exe且與執行檔在同一個工作目錄
    driver = webdriver.Chrome(chrome_options=options)
    driver.maximize_window()
    # 方法二：或是直接指定exe檔案路徑
    # driver = webdriver.Chrome(“桌面\chromedriver”)
    driver.get("https://web.pcc.gov.tw/prkms/tender/common/basic/indexTenderBasic") # 更改網址以前往不同網頁
    time.sleep(2)
#     driver.set_window_size(1920, 1080)
    # time.sleep(3)
except:
#     Warning('開啟瀏覽器失敗!')
    sys.exit()

C:\Users\GeorgeLin\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [23]:
input('完成查詢後按Enter繼續')

完成查詢後按Enter繼續


''

# 調整每頁顯示筆數

In [24]:
# 調整為每頁顯示100筆
# pagesize = driver.find_element_by_id("pageSizeSelector")
pagesize = driver.find_element(By.ID,"pageSizeSelector")
pagesize.send_keys('100')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="pageSizeSelector"]"}
  (Session info: chrome=100.0.4896.127)


# 撈取清單

In [ ]:
# soup = BeautifulSoup(driver.page_source, "html.parser")
# next_page = soup.find('a', text = '下一頁')
# next_page
# BeautifulSoup(driver.page_source, "html.parser").find('a', text = '下一頁')

In [ ]:
def get_case_list(website=website):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # soup
    table = soup.find('table', {'class': 'tb_01'})
    # table

    columns = [th.text.replace('\n', '').replace('\u2003', '') for th in table.find('tr').find_all('th')]
    columns

    trs = table.find_all('tr')[1:]
    rows = list()
    for tr in trs: #.replace('\xa0', '')
        rows.append([td.text.replace('\n', '').replace('\t', '').replace('!', '').replace(' ', '') for td in tr.find_all('td')])


    links = table.find_all(["div"], class_ = 'bt_cen1 no_blank')
    link_li = []
    for link in links:
        link_li.append(website+link.select_one("a").get("href"))

    df = pd.DataFrame(data = rows, columns = columns)
    df['連結'] = link_li
    time.sleep(1)
    return df 
    # df

In [ ]:
try:
    df = get_case_list()
    while BeautifulSoup(driver.page_source, "html.parser").find('a', text = '下一頁') != None:
        driver.find_element(By.LINK_TEXT,"下一頁").click()
        time.sleep(1)
        df = pd.concat([df, get_case_list()])
    print('共有{}筆資料'.format(len(df)))
    df
except:
    driver.quit()
    sys.exit()

In [ ]:
# driver.find_elements_by_xpath('//*[contains(text(), "下一頁")]')

# 撈取聯絡資訊

In [25]:
# driver = webdriver.Chrome()

In [32]:
def get_contact_info(link):
#     wait = WebDriverWait(driver,30)
#     driver = webdriver.Chrome()
    driver.get(link) # 更改網址以前往不同網頁
    try:
        while driver.find_element(By.XPATH,'//*[contains(text(), "驗證碼檢核")]') != None:
            time.sleep(2)
    except:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find('table', {'summary': '機關資料'})
        trs = table.find_all('tr')[1:]
        rows = list()
        for tr in trs:
            rows.append([td.text.replace('\n', '').replace('\t', '').replace('!', '').replace(' ', '') for td in tr.find_all('td')])
        info = pd.DataFrame(rows).set_index([0]).T
        info['連結'] = link
        time.sleep(1)
#     driver.quit()
        return info

In [33]:
try:
    info = pd.DataFrame()
    for i, link in enumerate(df['連結']):
        print(i)
        info = pd.concat([info, get_contact_info(link)])

except:
    # 避免中途中止
    result = pd.merge(df, info, on = '連結', how = 'left')
    result.to_excel('./政府電子採購網聯絡資訊{:%m%d}.xlsx'.format(datetime.datetime.today()), index = False)
#     result
    driver.quit()
    sys.exit()

0
1
2
3
4
5
6


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-32-5d4076d8bda3>", line 6, in get_contact_info
    while driver.find_element(By.XPATH,'//*[contains(text(), "驗證碼檢核")]') != None:
  File "C:\Users\GeorgeLin\.conda\envs\tensorflow\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "C:\Users\GeorgeLin\.conda\envs\tensorflow\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\GeorgeLin\.conda\envs\tensorflow\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.127)


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-33-2ef5f6294cb0>", line 5, in

TypeError: object of type 'NoneType' has no len()

In [ ]:
info

In [ ]:
# 輸出
result = pd.merge(df, info, on = '連結', how = 'left')
result.to_excel('./政府電子採購網聯絡資訊{:%m%d}.xlsx'.format(datetime.datetime.today()), index = False)
result

In [ ]:
driver.quit()

In [ ]:
# driver.get(df['連結'][0]) # 更改網址以前往不同網頁
# soup = BeautifulSoup(driver.page_source, "html.parser")
# # soup
# table = soup.find('table', {'summary': '機關資料'})
# # table
# trs = table.find_all('tr')[1:]
# rows = list()
# for tr in trs: #.replace('\xa0', '')
#     rows.append([td.text.replace('\n', '').replace('\t', '').replace('!', '').replace(' ', '') for td in tr.find_all('td')])

In [ ]:
# t = pd.DataFrame(rows).set_index([0]).T
# t